In [33]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [34]:
merged = pd.read_csv(path+'data/merged.csv')

In [35]:
df = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality', 'driver_wins_after_race', 'driver_standings_pos_after_race',  'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1)

In [36]:
### Removing sprint races

df = df[~
(
    ((df.season == 2021) & (df['round'].isin([10, 14, 19])))
    |
    ((df.season == 2022) & (df['round'].isin([4, 11, 21])))
)
]

In [37]:
driver_points_per_race = []
driver_points_from_race = []

constructor_points_per_race = []
constructor_points_from_race = []

for index, row in df.iterrows():
    driver_points_per_race.append(round(row.driver_points / row['round'], 2))
    driver_points_from_race.append(round(row.driver_points_after_race - row.driver_points, 1))

    constructor_points_per_race.append(round(row.constructor_points / row['round'], 2))
    constructor_points_from_race.append(round(row.constructor_points_after_race - row.constructor_points, 1))

In [38]:
df['driver_points_per_race'] = driver_points_per_race
df['driver_points_from_race'] = driver_points_from_race

df['constructor_points_per_race'] = constructor_points_per_race
df['constructor_points_from_race'] = constructor_points_from_race

In [39]:
df.query('season == 2014 & round == 1')

,season,round,circuit_id,driver,constructor,podium,driver_points_after_race,driver_points,driver_wins,driver_standings_pos,...,constructor_standings_pos,qualifying_pos,qual_time,stage,q_delta,starting_grid,driver_points_per_race,driver_points_from_race,constructor_points_per_race,constructor_points_from_race
0,2014,1,albert_park,nico_rosberg,mercedes,1,25.0,0,0,0,...,0,3,104.595,q3,0.000,3,0.0,25.0,0.0,25.0
1,2014,1,albert_park,kevin_magnussen,mclaren,2,18.0,0,0,0,...,0,4,105.745,q3,0.000,4,0.0,18.0,0.0,33.0
2,2014,1,albert_park,jenson_button,mclaren,3,15.0,0,0,0,...,0,11,104.437,q2,0.206,10,0.0,15.0,0.0,33.0
3,2014,1,albert_park,fernando_alonso,ferrari,4,12.0,0,0,0,...,0,5,105.819,q3,0.000,5,0.0,12.0,0.0,18.0
4,2014,1,albert_park,valtteri_bottas,williams,5,10.0,0,0,0,...,0,10,108.147,q3,3.916,15,0.0,10.0,0.0,10.0
5,2014,1,albert_park,nico_hulkenberg,force_india,6,8.0,0,0,0,...,0,7,106.030,q3,0.000,7,0.0,8.0,0.0,9.0
6,2014,1,albert_park,kimi_raikkonen,ferrari,7,6.0,0,0,0,...,0,12,104.494,q2,0.263,11,0.0,6.0,0.0,18.0
7,2014,1,albert_park,jean-eric_vergne,toro_rosso,8,4.0,0,0,0,...,0,6,105.864,q3,0.000,6,0.0,4.0,0.0,6.0
8,2014,1,albert_park,daniil_kvyat,toro_rosso,9,2.0,0,0,0,...,0,8,107.368,q3,0.000,8,0.0,2.0,0.0,6.0
9,2014,1,albert_park,sergio_perez,force_india,10,1.0,0,0,0,...,0,16,107.293,q2,3.062,16,0.0,1.0,0.0,9.0


In [40]:
grouped = df.groupby(['season', 'round'])
last_3_races = []

for index, group in grouped:
        for _, row in group.iterrows():
                round = index[1]
                rounds = list(range(round-1, round-4, -1))
                points = df.query(
                        'season ==@index[0] & round in @rounds & driver ==@row.driver'
                        ).agg({
                                'driver_points_from_race': sum
                                }).values[0]

                last_3_races.append(points)

In [41]:
grouped = df.groupby(['season', 'round'])
points_before_race = []

for index, group in grouped:
        for _, row in group.iterrows():
                round = index[1]
                query = df.query(
                        'season ==@index[0] & round ==@round-1 & driver ==@row.driver')

                points = query.driver_points_after_race
                if len(points) != 0:
                        points_before_race.append(points.values[0])
                else:
                        points_before_race.append(0.0)

In [42]:
df['driver_points_last_3_races'] = last_3_races
df['driver_points_before_race'] = points_before_race

In [43]:
testing = df[['season','round', 'driver', 'driver_points',  'driver_points_before_race', 'driver_points_from_race', 'driver_points_last_3_races']]

In [44]:
testing.query('season == 2021 & driver == "max_verstappen"')

,season,round,driver,driver_points,driver_points_before_race,driver_points_from_race,driver_points_last_3_races
2761,2021,1,max_verstappen,0,0.0,18.0,0.0
2780,2021,2,max_verstappen,18,18.0,25.0,18.0
2800,2021,3,max_verstappen,43,43.0,18.0,43.0
2820,2021,4,max_verstappen,61,61.0,19.0,61.0
2839,2021,5,max_verstappen,80,80.0,25.0,62.0
2874,2021,6,max_verstappen,105,105.0,0.0,62.0
2876,2021,7,max_verstappen,105,105.0,26.0,44.0
2894,2021,8,max_verstappen,131,131.0,25.0,51.0
2914,2021,9,max_verstappen,156,156.0,26.0,51.0
2962,2021,11,max_verstappen,185,0.0,2.0,51.0


In [45]:
df[['driver', 'constructor', 
        # 'podium',
    #    'driver_points_after_race', 'driver_wins_after_race',
    #    'driver_standings_pos_after_race', 'constructor_points_after_race',
    #    'constructor_wins_after_race', 'constructor_standings_pos_after_race',
    #    'qualifying_pos',
        # 'starting_grid',
    #    'qual_time', 
       'stage', 'q_delta']].groupby('stage').describe()

q_delta                                                      
        count      mean       std  min    25%    50%    75%     max
stage                                                              
q1      897.0  3.742524  3.144322  0.0  2.454  3.085  3.939  33.846
q2      901.0  2.293517  1.520097  0.0  1.625  2.129  2.570  26.141
q3     1705.0  1.246313  1.579176  0.0  0.319  0.932  1.593  22.112

In [46]:
df.to_csv(path+'data/processed.csv', index=False)